# **Hotspot Analysis:**

#### Import libraries

In [ ]:
import sys
sys.path.append("./HotspotAnalysis/")
import os
from AnalysisFunctions import *
from ROIFunctions import *
from PlottingFunctions import *
import cv2
import numpy as np
from imageio import imwrite
import holoviews as hv
from holoviews import opts
from skimage.measure import block_reduce
hv.notebook_extension('bokeh')
#from IPython.display import display, HTML
#display(HTML("""
#<style>
#.output {
#    display: flex;
#    align-items: center;
#    text-align: center;
#    }
#</style>
#"""))

# **Image Processing**

## 1. Load image & set image name

- ##### Write the file path to the folder where your image is located in *'imgPath'*
- ##### Write the image file name in *'imageName'*
- ##### Image names should be written with the following format: "MouseName_ExperimentalInfo_sectionX#D.ext", where: 
> ##### X is the row number of the section
> ##### # is the section number
> ##### D is the direction of the section ('L' or 'l' for left; 'R' or 'r' for right)
> ##### ext is the file extension (e.g. 'tif' or 'czi')
- ##### Write which channel is the channel to be analyzed in *'channelOfInterest'* (i.e. "green", "red")

In [ ]:
# imgPath = "/Users/joezaki/Downloads/CtxStrRGP97M/CtxStrRGP97M2/"
# imgName = "CtxStrRGP97M2_sectionC2_CalB_DAPI.czi"
imgPath = '/Volumes/Elements/S1_PFC_Adult1/'
imgName = 'S1_PFC_Adult1c_sectionC4R.czi'
savePath = None

img, DAPI = loadImage(imgPath, imgName, img_channel=1, DAPI_channel=0)
name = imgName.split(".")[0]
opts.defaults(
    opts.GridSpace(shared_xaxis=True, shared_yaxis=True),
    opts.Image(cmap='gist_gray', width=int(img.shape[1]/3), height=int(img.shape[0]/3)),
    opts.Labels(text_color='white', text_font_size='8pt', text_align='left', text_baseline='bottom'),
    opts.Path(color='white'),
    opts.Spread(width=600),
    opts.Overlay(show_legend=False))

##### Use the following chunk if you'd like to downsample your images:

In [ ]:
row_downsample = 2
col_downsample = 2
img = block_reduce(img, (row_downsample,col_downsample), np.sum)#.astype('uint8')
DAPI = block_reduce(DAPI, (row_downsample,col_downsample), np.sum)#.astype('uint8')

### Plots the image channel alongside the DAPI channel

In [ ]:
%output size=100
allChannels = np.dstack((DAPI, img))
ds = hv.Dataset((['DAPI','Image'], np.arange(DAPI.shape[1]), np.arange(DAPI.shape[0]), allChannels), ['Channel','X', 'Y'], 'Intensity')
allChannelsPlot = ds.to(hv.Image, ['X', 'Y'], dynamic=True)
allChannelsPlot.opts(invert_yaxis=True)

## 2. Remove saturated pixels

##### _Note_: Change threshold based on bit depth of image

In [ ]:
#%%output size=100
if type(img[0,0]) == np.uint8:
    threshold = 200
elif type(img[0,0]) == np.uint16:
    threshold = 50000
else:
    raise Exception("img is not 8- or 16-bit")
imgNew = img.copy()
avgPixel = np.mean(img)
imgNew[imgNew > threshold] = avgPixel
thresholded = np.dstack((img, imgNew))
ds = hv.Dataset((np.arange(2), np.arange(img.shape[1]), np.arange(img.shape[0]), thresholded), ['Threshold','X', 'Y'], 'Intensity')
thresholdedPlots = ds.to(hv.Image, ['X', 'Y'], dynamic=True)
thresholdedPlots.opts(invert_yaxis=True)

##### Assign thresholded image to img once you are satisfied with the saturated pixel removal

In [ ]:
img = imgNew

## 3. Draw striatum ROI or load pre-drawn ROI

##### Option 1: Double click to begin ROI, draw ROI with single clicks, end ROI with double click

In [ ]:
%output size=30
region=['ROI']
image,vertices = ROI_plot(cv2.convertScaleAbs(DAPI, alpha=1, beta=-100),region)
hv.ipython.display(image)

In [ ]:
ROI_masks, mask, croppedImg, maskedImage, ROIimage, processedImage = make_ROI(img, DAPI, region, vertices)
#del DAPI

##### Option 2: Supply an roiPath to where the ROI mask is located
>##### If no roiPath is supplied, it is assumed in the line below that the ROI lives in your `imgPath` in a folder named `ROIs` and is named as the `imgName` with `_ROI.tif` attached to the end

In [ ]:
roiPath = os.path.join(os.path.join(imgPath, 'ROIs'), '{}_ROI.tif'.format(imgName))
ROI_masks, mask, croppedImg, maskedImage, ROIimage, processedImage = load_ROI(img, DAPI, roiPath)

# **Image Analysis**
> #### The analysis is comparing your image to an image where the pixels are randomly distributed throughout the same area

## 4. Set neighborhood size & run analysis

In [ ]:
%%time
nx = 20
ny = 20
FULLname = name + "_" + str(nx) + "x" + str(ny)
stats = Getis(mask, maskedImage, nx, ny)
direction, zs, DL, VL, DM, VM, MLaxisZs, DVaxisZs, quadrantStds = processedStats(stats, maskedImage, FULLname)

## 5. Visualizations from Analyzed Image

In [ ]:
%%time
Heatmap = HeatmapPlot(zs)
DV_ML = DV_ML_Plot(stats, zs, MLaxisZs, DVaxisZs, direction)
Quadrant = QuadrantPlot(stats, maskedImage, DL, VL, DM, VM)
Gstat = statsPlot(stats, maskedImage, "Gstat")
Hotspot = statsPlot(stats, maskedImage, "Hotspot")
Zdistribution = ZdistributionPlot(stats, FULLname)

## 6. Create new folder, export statistics as CSV and visualizations as PDFs

In [ ]:
if savePath is None:
    savePath = imgPath

os.chdir(savePath)
path = "./" + FULLname
try:
    os.mkdir(path)
except OSError:
    print ("Creation of the directory %s failed" % path)
else:
    print ("Successfully created the directory %s " % path)
os.chdir(path)

exportStats = stats.to_csv((FULLname + "_GetisOrdStats.csv"))
imwrite((FULLname + "_ROI.png"), ROI_masks['ROI'].astype('uint8')*255)
Heatmap.savefig(((FULLname + "_HeatmapPlot.pdf")), bbox_inches="tight")
DV_ML.savefig(((FULLname + "_DV_ML_Plot.pdf")), bbox_inches="tight")
Quadrant.savefig(((FULLname + "_QuadrantPlot.pdf")), bbox_inches="tight")
Gstat.savefig(((FULLname + "_GstatPlot.pdf")), bbox_inches="tight")
Hotspot.savefig(((FULLname + "_HotspotPlot.pdf")), bbox_inches="tight")
Zdistribution.savefig(((FULLname + "_ZdistributionPlot.pdf")), bbox_inches="tight")
del exportStats, Heatmap, DV_ML, Quadrant, Gstat, Hotspot, Zdistribution